# Bienvenue sur le meilleur notebook

In [1]:
import numpy as np
from matplotlib import pyplot as plt

# Calculs de bornes primales

In [39]:
def computeCost(p,d,w,order):
    t=0
    cost=0
    for idx in order:
        t += p[idx]
        cost += max(0,(t-d[idx])*w[idx])
    return cost

In [86]:
def getPrimal_0(p,d,w):
    sortedTasks=np.argsort(d)
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,list(sortedTasks))

In [87]:
def getPrimal_1(p,d,w):
    sortedTasks=(np.argsort((np.array(d)-np.array(p))*np.array(w)))[::-1]
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,list(sortedTasks))

In [88]:
def getPrimal_2(p,d,w):
    tasksLeft=[int(i) for i in range(len(p))]
    order=[]
    while len(tasksLeft)!= 0:
        time=np.sum(np.array(p)[tasksLeft])
        penalities=(time-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
        sortedTasks=np.argsort(penalities)
        order.append(tasksLeft[sortedTasks[0]])
        tasksLeft.pop(sortedTasks[0])
    order.reverse()
    cost = computeCost(p,d,w,order)
    return(cost,order)

## Test des différentes méthodes : 

In [91]:
w=[4,5,3,5]
d=[16,26,25,27]
p=[12,8,15,9]
print("Classement par ordre croissant des dates d'usinage au plus tard :\n",getPrimal_0(p,d,w))
print("Classement par ordre décroissant des (d-p)*w : \n",getPrimal_1(p,d,w))
print("Classement en mettant en dernière position disponible l'élémen non traité donnant lieu à la plus petite pénalité de retard : \n",getPrimal_2(p,d,w))

Classement par ordre croissant des dates d'usinage au plus tard :
 (136, [0, 2, 1, 3])
Classement par ordre décroissant des (d-p)*w : 
 (133, [3, 1, 2, 0])
Classement en mettant en dernière position disponible l'élémen non traité donnant lieu à la plus petite pénalité de retard : 
 (67, [0, 1, 3, 2])


# Calculs de borne dual

In [127]:
def getDual_0(p,w,d,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    return dual

In [128]:
def getDual_1(p,w,d,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    for j in notVisited:
        retard=(p[j]+time)-d[j]
        dual+=retard*w[j]*(retard>0)
    return dual

## Test des méthodes : 

In [129]:
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : \n",getDual_0(p,w,d,[0,1]))
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : \n",getDual_1(p,w,d,[0,1]))

Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : 
 0
Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : 
 40


# Implementation 

In [2]:
class Node:
    def __init__(self, parent, children, lb, visited):
        self.parent = parent
        self.children = children
        self.lb = lb
        self.visited = visited
    def __str__(self):
        return "Parent : " +str(self.parent)+"\n Children : "+str(self.children) + "\n ub : "+str(self.ub)+"\n lb : "+str(self.lb) + "\n visited : "+str(slef.visited)

In [ ]:
def branch_and_bound(p,d,w):
    feascycles = enum_smallcycles(graphe, K) # generate cycles of size <= K 
    
    Tree = [] # list of all nodes created (a list of Node objects)
    Queue = [] # list of nodes to process (a list of integers with the index of nodes to process in the Tree)
    UB = 100000 # set the upper bound to a sufficiently large number
    LB = 0 # set the lower bound to a sufficiently small number 
    #(if a feasible solution is known we can use its value, here choosing no cycles is a trivial feasible solution)
    ϵ = 0.0001 #an optimality tolerance of %0.01
    incumbent = [] # initialize the incumbent solution 

    root = Node(None, [], math.inf, [], []) # at the root node no variables are fixed 
    Tree.append(root) # start the tree with the root node
    Queue.append(0) # start the list of nodes to process with the root node (note that index of root in Tree is 0)
    
    #continue processing nodes until the list is empty OR 
    #until an optimality tolerance between the lower bound and the upper bound is reached
    while Queue!=[] and 2*(UB-LB)/(UB+LB)>=ϵ:
        # process the first node in the queue
        currentindex = Queue[0] # note that we follow a first-in-first-out node processing strategy
        currentnode = Tree[currentindex]
        # solve the LP relaxation of the current node
        current_val,current_sol = plne_cycle_relax(graphe, utilities, feascycles, K, currentnode.setzero, currentnode.setone)
        #print(current_val)
        #print(current_sol) 
    
        # if the current node is feasible then we will proceed to updating the upper bound of the current node
        # and checking whether its solution is fractional
        if current_sol!=[]:
            currentnode.ub = current_val
            fractional = False # initialize the indicator for fractional solutions
            branchingindex = 0 # initialize the index of branching variable
            # test whether the current solution is fractional
            # TO COMPLETE
            for i in list(range(0,len(current_sol))):
                if (current_sol[i]-math.floor(current_sol[i]))>0:
                    branchingindex = i
                    fractional = True
                    break
            # note that we follow a first-found branching strategy, i.e., we branch on the first fractional variable found        
            # if the current solution is fractional and its bound is promising:
            # - create two new nodes 
            # - add them to the tree (set their parent as currentnode)
            # - add their indices to the list of children of current node
            # - add the new nodes to the Queue 
            if (fractional == True and (currentnode.ub-LB)>LB*ϵ): 
                print("Branching on variable cycle_", branchingindex)
                n1=Node(currentnode, [], currentnode.ub, currentnode.setzero.append(branchingindex), [])
                n2=Node(currentnode, [], currentnode.ub, [], currentnode.setzero.append(branchingindex))
                Tree.append(n1)
                Queue.append(len(Tree)-1)
                currentnode.children.append(len(Tree)-1)
                Tree.append(n2)
                Queue.append(len(Tree)-1)
                currentnode.children.append(len(Tree)-1)
                
                # TO COMPLETE :    
            # if the current solution is fractional but its bound is not promising then do nothing    
            elif (fractional == True and currentnode.ub<=LB):
                print("Node eliminated by bound")
            # if the current solution is integer update the lower bound and the incumbent solution (if necessary)   
            elif fractional == False: #Solution non fractionnelle
                print("Integer solution with value ", current_val, " found")
                if(currentnode.ub-LB>0):
                    LB=currentnode.ub
                    incumbent=current_sol
                # TO COMPLETE
        
        # Calculate the current UB as the maximum of all the upper bounds among the active nodes
        # MAJ de la borne dual (si pb de max, si on a un max plus petit on met a jour)
        currentUB = math.inf
        for node in Tree:
            if node.ub < currentUB:
                currentUB = node.ub
        
        # Update the UB if the current UB is better
        if currentUB <= UB:
            UB = currentUB
        # Display the current upper and lower bounds    
        print("LB= ",LB," UB= ", UB)
        # When the processing of the node is completed remove the node from the queue     
        Queue.remove(currentindex)


    # return the incumbent solution 
    C = list(range(0,len(feascycles)))
    bb_cycles = []  
    for c in C:
        if incumbent[c] == 1:
            bb_cycles.append(feascycles[c])
    
    return bb_cycles